In [1]:
using NearestNeighbors
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Base.Collections
using WordStreams
using DataStructures
using Clustering
using Distances
using Iterators
using StatsBase
using Training

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/PooledElements/src/pooledstring.jl:480
  likely near /home/ubuntu/.julia/v0.5/PooledElements/src/pooledstring.jl:480
  likely near /home/ubuntu/.julia/v0.5/PooledElements/src/pooledstring.jl:480
  likely near /home/ubuntu/.julia/v0.5/PooledElements/src/pooledstring.jl:480
in Type at /home/ubuntu/.julia/v0.5/PooledElements/src/pool.jl
  likely near /mnt/phd/prototypes/SenseSplittingWord2Vec/src/WordDistributions.jl:2
  likely near /mnt/phd/prototypes/SenseSplittingWord2Vec/src/WordDistributions.jl:2
  likely near /mnt/phd/prototypes/SenseSplittingWord2Vec/src/WordDistributions.jl:2

Use "Dict(a=>b for (a,b) in c)" instead.


In [ ]:
ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_300_i1.model");
dtree,labels = nn_tree(ee)
""

In [ ]:
@time nears=find_nearest_words(ee,"states",dtree,labels )

In [ ]:
trained_times = collect(values(ee.trained_times["states"]))
items = sort(collect(zip(trained_times, nears)))
map(println, items)

In [ ]:
function load_wordsim353(filename="data/corpora/wordsim353/combined.csv")
    wordsims = readdlm(filename, ','; skipstart=1)
    wordpairs = convert(Matrix{String},wordsims[:,1:2])
    groundsim = convert(Vector{Float64},wordsims[:,3])
    (wordpairs, groundsim)
end
wordpairs, groundsim = load_wordsim353()

In [ ]:
function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String})
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            wvs_1 = hcat(ee.embedding[wordpairs[ii,1]]...)
            wvs_2 = hcat(ee.embedding[wordpairs[ii,2]]...)
            sims[ii]=1-minimum(pairwise(CosineDist(), wvs_1,wvs_2))
        catch ex
            warn(ex)
            sims[ii]=NaN
        end
    end
    return sims
end
sims = get_sims(ee,map(lowercase,wordpairs))
corspearman(groundsim,sims)

In [ ]:
function load_SCWS(filename = "data/corpora/SCWS/ratings.txt")
    function get_context2(fulltext)
        Task() do
            skip_next = 0
            for word in split(fulltext, ' ')
                skip_next-=1
                if word == "<b>"
                    skip_next=3 
                end
                skip_next>0 && continue
                produce(lowercase(word))
            end
        end |> collect
    end
    entries = readdlm(filename, '\t', String; quotes=false, comments=false)
    wordpairs = map(lowercase,entries[:,[2,4]])
    groundsim = map(s->parse(Float64,s), entries[:,8])
    contexts = map(get_context2,entries[:,[6,7]])
    (wordpairs,contexts,groundsim,)
end
wordpairs,contexts,groundsim = load_SCWS();
""

In [ ]:
function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String}, contexts::Matrix)
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            
            sense1 = Training.WSD(ee, wordpairs[ii,1],contexts[ii,1];skip_oov=true)
            wv1 = ee.embedding[wordpairs[ii,1]][sense1]
            
            sense2 = Training.WSD(ee, wordpairs[ii,2],contexts[ii,2];skip_oov=true)
            wv2 = ee.embedding[wordpairs[ii,2]][sense2]
            
            sims[ii]=1-cosine_dist(wv1,wv2)
        catch ex
            if typeof(ex)==KeyError
                warn(ex)
                sims[ii]=NaN
            else
                rethrow(ex)
            end
        end
    end
    return sims
end
local_sims = get_sims(ee,wordpairs,contexts)
corspearman(groundsim,local_sims)

In [13]:
corspearman(groundsim,local_sims)


0.3941527198150464

In [14]:

function weighted_average(logprobs, embeddings)
    function normal_probs(logprobs::Vector)
        ret = copy(logprobs)
        max_lp = maximum(logprobs)
        ret.-=max_lp #Bring closer to zero
        map!(exp,ret)
        denom = sum(ret)
        ret./=denom
    end
    
    
    ret = zeros(size(embeddings[1]))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight*embedding
    end
    ret
end

function all_logprobs_of_context(embed, context, middle_word)
    [Query.logprob_of_context(embed, context,wv; skip_oov=true, normalise_over_length=true)
        for wv in embed.embedding[middle_word]]
end

function synthesize_embedding(ee,context, middle_word)
    logprobs = all_logprobs_of_context(ee, context,middle_word)
    weighted_average(logprobs, ee.embedding[middle_word])
end

function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String}, contexts::Matrix)
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try           
            wv1 = synthesize_embedding(ee, contexts[ii,1],wordpairs[ii,1])
            wv2 = synthesize_embedding(ee, contexts[ii,2],wordpairs[ii,2])
             
            
            sims[ii]=1-cosine_dist(wv1,wv2)
        catch ex
            if typeof(ex)==KeyError
                warn(ex)
                sims[ii]=NaN
            else
                rethrow(ex)
            end
        end
    end
    return sims
end
local_sims = get_sims(ee,wordpairs,contexts)
corspearman(groundsim,local_sims)

LoadError: LoadError: DimensionMismatch("arrays could not be broadcast to a common size")
while loading In[14], in expression starting on line 50

In [ ]:
?Query.logprob_of_context

In [ ]:
describe(collect(values(ee.distribution)))